### --- Day 22: Monkey Map ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/22

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 22);
WriteLines(inputLines);

Loading puzzle file: Day22.txt
Total lines: 202
Max line length: 5610

                                                  ........#..................#...#....#.........#................................#.....#........#.....
                                                  #.......#.......#................#.#....#..............#..#.......#............#...#.......#.......#
                                                  ...........#...#.....##............#.#.....##.......#....#.........#.............#........#....#.#..
                                                  .............................#.......##.....#.....#........#....#..#.#..............................
                                                  ...........#..................##...#..................#..........#.......#...#.....#..#.............


In [4]:
string[] testInputLines = 
[
    "        ...#",
    "        .#..",
    "        #...",
    "        ....",
    "...#.......#",
    "........#...",
    "..#....#....",
    "..........#.",
    "        ...#....",
    "        .....#..",
    "        .#......",
    "        ......#.",
    "",
    "10R5L5R10L4R5L5",
];

In [5]:
using PointChar = (Point point, char ch);
using PointDir = (Point pos, Point dir);
const char Wall = '#';
const char Blank = ' ';

int FindPassword(string[] inputLines)
{
    var (gridLines, stepLines) = inputLines.SeparateBy(l => l is "").ToArray();

    CharGrid grid = new(gridLines);
    
    var stepLine = stepLines[0];
    Regex stepRegex = new("(\\d+)|R|L");
    var stepMatch = stepRegex.Matches(stepLine);

    // You begin the path in the leftmost open tile of the top row of tiles.
    // Initially, you are facing to the right (from the perspective of how the
    // map is drawn).

    var horizLookup = GetLookups(pch => pch.point.Y, pch => pch.point.X);
    var vertLookup = GetLookups(pch => pch.point.X, pch => pch.point.Y);

    var startRow = horizLookup[0];
    var startPoint = startRow[0].point;
    var startDir = Right;

    PointDir current = (startPoint, startDir);
    foreach (Match m in stepMatch)
    {
        var instr = m.Groups[0].Value;

        if (int.TryParse(instr, out var steps))
        {
            current = Walk(current, steps);
        }
        else
        {
            var nextDir = instr switch 
            {
                "L" => TurnLeft(current.dir),
                "R" => TurnRight(current.dir),
                _ => throw new Exception("Unexpected direction")
            };
            current = (current.pos, nextDir);
        }
    }

    var result = CalcPassword(current);
    return result;

    PointDir Walk(PointDir current, int steps)
    {
        var pathLookup = current.dir.X is not 0 ? horizLookup : vertLookup;
        var singlePath = current.dir switch
        {
            (0, -1) => vertLookup[current.pos.X].Reverse(),
            (0, 1) => vertLookup[current.pos.X],
            (-1, 0) => horizLookup[current.pos.Y].Reverse(),
            (1, 0) => horizLookup[current.pos.Y],
            _ => throw new Exception("Unexpected direction")
        };

        var repeatingPath = Range(0, steps).SelectMany(_ => singlePath);
        var stopPoint = repeatingPath
                        .SkipWhile(p => p.point != current.pos)
                        .Take(steps + 1)
                        .TakeWhile(pch => pch.ch is not Wall)
                        .Last();

        return (stopPoint.point, current.dir);
    }

    Dictionary<int, PointChar[]> GetLookups(Func<PointChar, int> getDim, Func<PointChar, int> getOtherDim)
    {
        var x = grid.Enumerate()
                    .Where(pch => pch.ch is not Blank)
                    .GroupBy(pch => getDim(pch))
                    .ToDictionary(g => g.Key, g => g.OrderBy(getOtherDim).ToArray());
        return x;
    }
}

int CalcPassword(PointDir current)
{
    Dictionary<Point, int> dirPointSpec = new() {
        [Right] = 0,
        [Down] = 1,
        [Left] = 2,
        [Up] = 3
    };
    var dirPoint = dirPointSpec[current.dir];
    var rowPoint = (current.pos.Y + 1) * 1000;
    var colPoint = (current.pos.X + 1) * 4;

    return rowPoint + colPoint + dirPoint;
}

In [6]:
var testAnswer = FindPassword(testInputLines);
Console.WriteLine(testAnswer);

6032


In [7]:
var part1Answer = FindPassword(inputLines);
Console.WriteLine(part1Answer);

191010


In [8]:
// 191010 is correct!
Ensure(191010, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/22

Ok, for part 2, we have to stitch panels into a cube! So we have two immediate challenges: how to decompose the original grid into 50x50 panels, and then, how to travel around the cube.

For the latter, I was thinking we could imagine ourselves as staring at a face of the cube. We know which panels will be above, below, left, right, and behind. As we travel off one side, we need to rotate the cube, eg, if we move off the top, the top panel becomes the main face - we are basically rotating the cube down:

* The behind panel becomes the new top
* The old top becomes the new face
* The old face becomes the new bottom
* The left / right panels remain the same, but are transformed
  * The left panel is rotated 90 degrees clockwise (LHS goes to top)
  * The right panel is rotated 90 degrees anti-clockwise (LHS goes to bottom)

Retrospective note: having made a start on the above, I decided to change strategy...

The challenge with panel faces is that the initial state still needs to be calculated, that is, some panels need to be rotated into the right configuration. So, this approach still requires us to pre-construct our cube to figure out which sides are adjacent. Therefore, given no saving in "geometric effort", I decided to focus on the edges themselves. For any given arrangement of panels there are fourteen open edges that need to be joined. We can join them in pairs, giving us seven pairs of edges to calculate. Once we know the edges, we can create virtual "warps" such that when the cursor walks off the edge of a panel, we can warp the cursor to the adjacent panel, facing the inside direction. Figuring out the edges is a bit fiddly, but once we have those, the advantage is that the cursor moves around on the flat map in the original orientation, making its path easy to understand.

For the sample input and puzzle input, I drew the tiles on paper, cut them out, and folded them into cubes! Quite cheesy, but it minimised mistakes. I'll be fascinated to see what other approaches were available!

In [10]:
using WarpSpec = (PointDir origin, PointDir destination, bool invertDest);

// First, introduce x/y regions for each panel, so we can link the edges

// From the puzzle description:

//         1111
//         1111
//         1111
//         1111
// 222233334444
// 222233334444
// 222233334444
// 222233334444
//         55556666
//         55556666
//         55556666
//         55556666

Point[] testInputRegions = 
[
    (2, 0),
    (0, 1),
    (1, 1),
    (2, 1),
    (2, 2),
    (3, 2)
];

// The output of my handicraft :) The spec contains for each region, the
// "exiting" side, and the corresponding "entering" side. The invertDest
// parameter represents that for some edges, we may, eg: exit at the top of one
// panel but enter at the bottom.
WarpSpec[] testInputWarps = 
[
    // a
    (((2, 1), Right), ((3, 2), Up), invertDest: true),
    // b
    (((2, 0), Right), ((3, 2), Right), invertDest: true),
    // c
    (((2, 0), Left), ((1, 1), Up), false),
    // d
    (((2, 0), Up), ((0, 1), Up), invertDest: true),
    // e
    (((1, 1), Down), ((2, 2), Left), invertDest: true),
    // f
    (((2, 2), Down), ((0, 1), Down), invertDest: true),
    // g
    (((0, 1), Left), ((3, 2), Down), invertDest: true),
];

int FindPassword2(string[] inputLines, int regionSize, Point[] regions, WarpSpec[] warpSpecs)
{
    var (gridLines, stepLines) = inputLines.SeparateBy(l => l is "").ToArray();

    CharGrid grid = new(gridLines);

    Dictionary<PointDir, PointDir> warps = new();
    foreach (var warpSpec in warpSpecs)
    {
        AddWarp(warpSpec.origin, warpSpec.destination, warpSpec.invertDest);
    }
    
    TestRegions();

    var startPoint = (inputLines[0].IndexOf("."), 0);
    var startDir = Right;

    var stepLine = stepLines[0];
    Regex stepRegex = new("(\\d+)|R|L");
    var stepMatch = stepRegex.Matches(stepLine);

    PointDir current = (startPoint, startDir);
    foreach (Match m in stepMatch)
    {
        var instr = m.Groups[0].Value;

        if (int.TryParse(instr, out var steps))
        {
            current = Walk(current, steps)
                        .TakeWhile(pd => grid[pd.pos] is not Wall)
                        .Last();
        }
        else
        {
            var nextDir = instr switch 
            {
                "L" => TurnLeft(current.dir),
                "R" => TurnRight(current.dir),
                _ => throw new Exception("Unexpected direction")
            };
            current = (current.pos, nextDir);
        }
    }

    var result = CalcPassword(current);
    return result;

    void AddWarp(PointDir originRegion, PointDir destRegion, bool invertDest)
    {
        Point originMin = originRegion.pos * regionSize;
        Point originSide = originRegion.dir;
        var originEdge = GetEdge(originMin, originSide).ToList();

        Point destMin = destRegion.pos * regionSize;
        Point destSide = destRegion.dir;
        var destEdge = GetEdge(destMin, destSide).ToList();
        if (invertDest) { destEdge.Reverse(); }

        foreach (var (from, to) in originEdge.Zip(destEdge))
        {
            var fromEdgePoint = from + originRegion.dir;
            warps[(fromEdgePoint, originRegion.dir)] = (to, Reverse(destRegion.dir));

            var toEdgePoint = to + destRegion.dir;
            warps[(toEdgePoint, destRegion.dir)] = (from, Reverse(originRegion.dir));
        }
    }

    IEnumerable<Point> GetEdge(Point min, Point dir)
    {
        var yTop = min.Y;
        var yBottom = min.Y + regionSize - 1;
        var xLeft = min.X;
        var xRight = min.X + regionSize - 1;

        return Range(0, regionSize).Select(i => dir switch 
        {
            // Top
            (0, -1) => new Point(xLeft + i, yTop),
            // Right
            (1, 0) => new Point(xRight, yTop + i),
            // Left
            (-1, 0) => new Point(xLeft, yTop + i),
            // Bottom
            (0, 1) => new Point(xLeft + i, yBottom),
            _ => throw new Exception("Unexpected")
        });
    }

    IEnumerable<PointDir> Walk(PointDir start, int steps)
    {
        var current = start;

        foreach (var _ in Range(0, steps))
        {
            yield return current;

            PointDir next = (current.pos + current.dir, current.dir);
            if (warps.TryGetValue(next, out var warped))
            {
                next = warped;
            }
            current = next;
        }
        
        yield return current;
    }

    void TestRegions()
    {
        var expectedPoints = 14 * regionSize;
        if (warps.Count != expectedPoints)
        {
            // Not enough edges! Perhaps we mistakenly double-counted an edge
            throw new Exception($"Expected {expectedPoints}, got {warps.Count}");
        }

        // From each region (panel), we should be able to walk in a straight
        // line and return back to our origin. Do this travelling horizontally /
        // vertically for all panels, to confirm we have walked over every edge
        
        foreach (var region in regions)
        {
            var startPoint = region * regionSize;
            var testDirections = List(Down, Right);
            foreach (var dir in testDirections)
            {
                var endPoint = Walk((startPoint, dir), 4 * regionSize).Last();
                if (endPoint.pos != startPoint)
                {
                    Console.WriteLine($"Failed on region {region}, direction {dir}. Expected point {startPoint}, got {endPoint}");
                    throw new Exception("Failed test. Check this");
                }
            }
        }
    }
}

Point Reverse(Point dir) => TurnRight(TurnRight(dir));

In [11]:
// ...In this example, the final row is 5, the final column is 7, and the final
// facing is 3, so the final password is 1000 * 5 + 4 * 7 + 3 = 5031.

var part2TestResult = FindPassword2(testInputLines, regionSize: 4, testInputRegions, testInputWarps);
Console.WriteLine(part2TestResult);

5031


In [12]:
// And now for the puzzle input

Point[] inputRegions = 
[
    (1, 0),
    (2, 0),
    (1, 1),
    (0, 2),
    (1, 2),
    (0, 3)
];

WarpSpec[] inputWarps = 
[
    // a
    (((1, 1), Right), ((2, 0), Down), false),
    // b
    (((1, 2), Right), ((2, 0), Right), invertDest: true),
    // c
    (((1, 1), Left), ((0, 2), Up), false),
    // d
    (((1, 0), Left), ((0, 2), Left), invertDest: true),
    // e
    (((1, 0), Up), ((0, 3), Left), false),
    // f
    (((1, 2), Down), ((0, 3), Right), false),
    // g
    (((2, 0), Up), ((0, 3), Down), false)
];

// Fold the map into a cube, then follow the path given in the monkeys' notes.
// What is the final password?
var part2Answer = FindPassword2(inputLines, regionSize: 50, inputRegions, inputWarps);
Console.WriteLine(part2Answer);

55364


In [13]:
// 55364 is correct!
Ensure(55364, part2Answer);